# Machine Learning Language Translation

In [7]:
!pip install --upgrade tensorflow

  Using cached tensorflow-2.19.0-cp312-cp312-win_amd64.whl.metadata (4.1 kB)
Using cached tensorflow-2.19.0-cp312-cp312-win_amd64.whl (376.0 MB)
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.18.0
    Uninstalling tensorflow-2.18.0:
      Successfully uninstalled tensorflow-2.18.0
  Rolling back uninstall of tensorflow
  Moving to c:\users\alok3\anaconda3\lib\site-packages\tensorflow-2.18.0.dist-info\
   from C:\Users\alok3\anaconda3\Lib\site-packages\~ensorflow-2.18.0.dist-info


ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\alok3\\anaconda3\\Lib\\site-packages\\tensorflow\\compiler\\mlir\\lite\\python\\_pywrap_converter_api.pyd'
Consider using the `--user` option or check the permissions.



In [9]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense

In [10]:
# Training parameters
batch_size = 64         # Batch size for training
epochs = 100            # Number of epochs to train for
latent_dim = 256        # Latent dimensionality of the encoding space
num_samples = 10000     # Number of samples to train on
data_path = 'fra.txt'  # Path to the English-French translation data file


'fra.txt'

In [11]:
# 1. Initialize lists and sets
input_texts = []            # List to store English sentences
target_texts = []           # List to store French sentences (with start/end tokens)
input_characters = set()    # Set to hold unique characters in English
target_characters = set()   # Set to hold unique characters in French


In [14]:
# Read and process the data file
with open(data_path, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')

# Process each line up to the specified number of samples
for line in lines[:min(num_samples, len(lines) - 1)]:
    # Split each line into input and target text
    input_text, target_text, _ = line.split('\t')
    
    # Add start and end sequence tokens to target text
    # '\t' as start sequence, '\n' as end sequence
    target_text = '\t' + target_text + '\n'
    
    # Store the texts
    input_texts.append(input_text)
    target_texts.append(target_text)
    
    # Collect unique characters from input text
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    
    # Collect unique characters from target text
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)


In [15]:
# Convert character sets to sorted lists
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])


In [16]:
print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of samples: 10000
Number of unique input tokens: 70
Number of unique output tokens: 91
Max sequence length for inputs: 14
Max sequence length for outputs: 59


In [17]:
input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])

In [20]:
input_token_index

{' ': 0,
 '!': 1,
 '"': 2,
 '$': 3,
 '%': 4,
 '&': 5,
 "'": 6,
 ',': 7,
 '-': 8,
 '.': 9,
 '0': 10,
 '1': 11,
 '2': 12,
 '3': 13,
 '5': 14,
 '7': 15,
 '8': 16,
 '9': 17,
 ':': 18,
 '?': 19,
 'A': 20,
 'B': 21,
 'C': 22,
 'D': 23,
 'E': 24,
 'F': 25,
 'G': 26,
 'H': 27,
 'I': 28,
 'J': 29,
 'K': 30,
 'L': 31,
 'M': 32,
 'N': 33,
 'O': 34,
 'P': 35,
 'Q': 36,
 'R': 37,
 'S': 38,
 'T': 39,
 'U': 40,
 'V': 41,
 'W': 42,
 'Y': 43,
 'a': 44,
 'b': 45,
 'c': 46,
 'd': 47,
 'e': 48,
 'f': 49,
 'g': 50,
 'h': 51,
 'i': 52,
 'j': 53,
 'k': 54,
 'l': 55,
 'm': 56,
 'n': 57,
 'o': 58,
 'p': 59,
 'q': 60,
 'r': 61,
 's': 62,
 't': 63,
 'u': 64,
 'v': 65,
 'w': 66,
 'x': 67,
 'y': 68,
 'z': 69}

In [21]:
target_token_index

{'\t': 0,
 '\n': 1,
 ' ': 2,
 '!': 3,
 '%': 4,
 '&': 5,
 "'": 6,
 ',': 7,
 '-': 8,
 '.': 9,
 '0': 10,
 '1': 11,
 '2': 12,
 '3': 13,
 '5': 14,
 '8': 15,
 '9': 16,
 ':': 17,
 '?': 18,
 'A': 19,
 'B': 20,
 'C': 21,
 'D': 22,
 'E': 23,
 'F': 24,
 'G': 25,
 'H': 26,
 'I': 27,
 'J': 28,
 'K': 29,
 'L': 30,
 'M': 31,
 'N': 32,
 'O': 33,
 'P': 34,
 'Q': 35,
 'R': 36,
 'S': 37,
 'T': 38,
 'U': 39,
 'V': 40,
 'W': 41,
 'Y': 42,
 'a': 43,
 'b': 44,
 'c': 45,
 'd': 46,
 'e': 47,
 'f': 48,
 'g': 49,
 'h': 50,
 'i': 51,
 'j': 52,
 'k': 53,
 'l': 54,
 'm': 55,
 'n': 56,
 'o': 57,
 'p': 58,
 'q': 59,
 'r': 60,
 's': 61,
 't': 62,
 'u': 63,
 'v': 64,
 'w': 65,
 'x': 66,
 'y': 67,
 'z': 68,
 '\xa0': 69,
 '«': 70,
 '»': 71,
 'À': 72,
 'Ç': 73,
 'É': 74,
 'Ê': 75,
 'à': 76,
 'â': 77,
 'ç': 78,
 'è': 79,
 'é': 80,
 'ê': 81,
 'î': 82,
 'ï': 83,
 'ô': 84,
 'ù': 85,
 'û': 86,
 'œ': 87,
 '\u2009': 88,
 '’': 89,
 '\u202f': 90}

In [22]:
input_token_index,target_token_index

({' ': 0,
  '!': 1,
  '"': 2,
  '$': 3,
  '%': 4,
  '&': 5,
  "'": 6,
  ',': 7,
  '-': 8,
  '.': 9,
  '0': 10,
  '1': 11,
  '2': 12,
  '3': 13,
  '5': 14,
  '7': 15,
  '8': 16,
  '9': 17,
  ':': 18,
  '?': 19,
  'A': 20,
  'B': 21,
  'C': 22,
  'D': 23,
  'E': 24,
  'F': 25,
  'G': 26,
  'H': 27,
  'I': 28,
  'J': 29,
  'K': 30,
  'L': 31,
  'M': 32,
  'N': 33,
  'O': 34,
  'P': 35,
  'Q': 36,
  'R': 37,
  'S': 38,
  'T': 39,
  'U': 40,
  'V': 41,
  'W': 42,
  'Y': 43,
  'a': 44,
  'b': 45,
  'c': 46,
  'd': 47,
  'e': 48,
  'f': 49,
  'g': 50,
  'h': 51,
  'i': 52,
  'j': 53,
  'k': 54,
  'l': 55,
  'm': 56,
  'n': 57,
  'o': 58,
  'p': 59,
  'q': 60,
  'r': 61,
  's': 62,
  't': 63,
  'u': 64,
  'v': 65,
  'w': 66,
  'x': 67,
  'y': 68,
  'z': 69},
 {'\t': 0,
  '\n': 1,
  ' ': 2,
  '!': 3,
  '%': 4,
  '&': 5,
  "'": 6,
  ',': 7,
  '-': 8,
  '.': 9,
  '0': 10,
  '1': 11,
  '2': 12,
  '3': 13,
  '5': 14,
  '8': 15,
  '9': 16,
  ':': 17,
  '?': 18,
  'A': 19,
  'B': 20,
  'C': 21,
  'D'

In [23]:
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')

decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

In [25]:
# Vectorize the input and target texts
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    # Vectorize encoder input (one-hot encoding)
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    # Pad remaining timesteps with space character
    encoder_input_data[i, t + 1:, input_token_index[' ']] = 1.
    
    # Vectorize decoder input and target
    for t, char in enumerate(target_text):
        # Decoder input (one-hot encoding)
        decoder_input_data[i, t, target_token_index[char]] = 1.
        
        # Decoder target (shifted by one timestep)
        if t > 0:
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.
        
        # Pad remaining timesteps with space character
        decoder_input_data[i, t + 1:, target_token_index[' ']] = 1.
        decoder_target_data[i, t:, target_token_index[' ']] = 1.



In [26]:

print("Encoder input shape for first sample:", encoder_input_data[0].shape)

Encoder input shape for first sample: (14, 70)


In [28]:
# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard encoder_outputs and only keep the states.
encoder_states = [state_h, state_c]


In [29]:
 # Set up the decoder, using encoder_states as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                        initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [30]:
# Define the model that will turn
# encoder_input_data & decoder_input_data into decoder_target_data
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Run training
model.compile(optimizer='rmsprop', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)

Epoch 1/100


C:\Users\alok3\anaconda3\Lib\site-packages\keras\src\models\functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['keras_tensor_9', 'keras_tensor_13']. Received: the structure of inputs=('*', '*')
  warnings.warn(


125/125 ━━━━━━━━━━━━━━━━━━━━ 37s 263ms/step - accuracy: 0.9408 - loss: 2.3161 - val_accuracy: 0.9831 - val_loss: 2.2693
Epoch 2/100
125/125 ━━━━━━━━━━━━━━━━━━━━ 32s 259ms/step - accuracy: 0.9831 - loss: 2.1282 - val_accuracy: 0.9831 - val_loss: 2.4405
Epoch 3/100
125/125 ━━━━━━━━━━━━━━━━━━━━ 32s 257ms/step - accuracy: 0.9831 - loss: 2.3243 - val_accuracy: 0.9831 - val_loss: 2.6207
Epoch 4/100
125/125 ━━━━━━━━━━━━━━━━━━━━ 32s 256ms/step - accuracy: 0.9831 - loss: 2.5287 - val_accuracy: 0.9831 - val_loss: 2.8087
Epoch 5/100
125/125 ━━━━━━━━━━━━━━━━━━━━ 32s 256ms/step - accuracy: 0.9831 - loss: 2.7352 - val_accuracy: 0.9831 - val_loss: 3.0041
Epoch 6/100
125/125 ━━━━━━━━━━━━━━━━━━━━ 32s 257ms/step - accuracy: 0.9831 - loss: 2.9283 - val_accuracy: 0.9831 - val_loss: 3.2041
Epoch 7/100
125/125 ━━━━━━━━━━━━━━━━━━━━ 32s 258ms/step - accuracy: 0.9831 - loss: 3.1575 - val_accuracy: 0.9831 - val_loss: 3.3992
Epoch 8/100
125/125 ━━━━━━━━━━━━━━━━━━━━ 32s 253ms/step - accuracy: 0.9831 - loss: 3.412

In [5]:
# Define sampling models
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

# Reverse-Lookup token index to decode sequences back to something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())

def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length or find stop character.
        if (sampled_char == '\n' or 
            len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

# Test the decoder
for seq_index in range(100):
    # Take one sequence (part of the training set) for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

NameError: name 'Model' is not defined